In [ ]:
!pip install faiss-cpu pandas pyarrow huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="abhijit26/movie-vector-galaxy",
    repo_type="dataset",
    local_dir="data_full",
    local_dir_use_symlinks=False
)

In [ ]:
import os
import pandas as pd
import numpy as np
import faiss

In [ ]:
DATA_DIR = "data_full"

os.listdir(DATA_DIR)

## Metadata - Movie Information

In [ ]:
meta = pd.read_parquet(f"{DATA_DIR}/metadata.parquet")

In [ ]:
print(f"meta shape {meta.shape}")
meta.columns.tolist()

In [ ]:
meta.head(3)

In [ ]:
meta.info(memory_usage="deep")

In [ ]:
meta.isnull().sum().sort_values(ascending=False).head(18)

In [ ]:
meta.describe(include="all")

## Embeddings - Stores Semantic meanning

In [ ]:
emb = np.load(f"{DATA_DIR}/embeddings.npy")

In [ ]:
emb.shape

In [ ]:
emb.dtype

In [ ]:
emb.nbytes / (1024**3)

In [ ]:
emb[0][:10]

In [ ]:
np.linalg.norm(emb[0])

## FAISS index - fast search
This file is built FROM embeddings.
faiss.swigfaiss_avx2.IndexFlatIP type of faiss

In [ ]:
index = faiss.read_index(f"{DATA_DIR}/faiss_index.faiss")

In [ ]:
type(index)

In [ ]:
index.ntotal

In [ ]:
index.d

In [ ]:
movie_idx = 100

D, I = index.search(emb[movie_idx].reshape(1, -1), 5)

meta.iloc[I[0]][["title"]]

## development dataset creation

In [ ]:
DATA_FULL = "data_full"
DATA_DEV = "data_dev"

os.makedirs(DATA_DEV, exist_ok=True)

meta = pd.read_parquet(f"{DATA_FULL}/metadata.parquet")
emb = np.load(f"{DATA_FULL}/embeddings.npy").astype("float32")

In [ ]:
meta["imdb_votes"] = meta["imdb_votes"].fillna(0)
meta["imdb_votes"].describe()

In [ ]:
DEV_SIZE = 20000

top_meta = (
    meta.sort_values("imdb_votes", ascending=False)
        .head(DEV_SIZE)
)

In [ ]:
# to keep embedding align
indices = top_meta.index.to_numpy()
indices.sort()

In [ ]:
meta_dev = meta.iloc[indices].reset_index(drop=True)

meta_dev.to_parquet(f"{DATA_DEV}/metadata.parquet")

meta_dev.shape

In [ ]:
emb_dev = emb[indices]

np.save(f"{DATA_DEV}/embeddings.npy", emb_dev)

emb_dev.shape

In [ ]:
dim = emb_dev.shape[1]

index_dev = faiss.IndexFlatIP(dim)
index_dev.add(emb_dev)

faiss.write_index(index_dev, f"{DATA_DEV}/faiss_index.faiss")

In [ ]:
print(len(meta_dev))
print(emb_dev.shape[0])
print(index_dev.ntotal)

In [ ]:
movie_idx = 100

D, I = index_dev.search(
    emb_dev[movie_idx].reshape(1, -1),
    5
)

meta_dev.iloc[I[0]][["title", "imdb_votes"]]

In [ ]:
DATA_DEV = "data_dev"

meta = pd.read_parquet(f"{DATA_DEV}/metadata.parquet")
emb = np.load(f"{DATA_DEV}/embeddings.npy")
index = faiss.read_index(f"{DATA_DEV}/faiss_index.faiss")

print("Loaded:", len(meta))

In [ ]:
title_to_idx = {
    title.lower(): idx
    for idx, title in enumerate(meta["title"])
}

def recommend_similar(movie_title, k=5):

    movie_title = movie_title.lower()

    if movie_title not in title_to_idx:
        print("Movie not found!")
        return

    idx = title_to_idx[movie_title]

    query_vec = emb[idx].reshape(1, -1)

    # +1 because first result is the movie itself
    D, I = index.search(query_vec, k + 1)

    results = meta.iloc[I[0][1:]][[
        "title",
        "year",
        "imdb_votes"
    ]]

    return results.reset_index(drop=True)

recommend_similar("Batman")